In [ ]:
%pip install numpy
%pip install pandas
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from ahc import AHC
from kmeans import K_Means

np.random.seed(5)

Carregando dataset Iris: https://archive.ics.uci.edu/dataset/53/iris

In [ ]:
# dataset_path = input("Dataset path: ")
dataset_path = "./iris.csv"
data = pd.read_csv(dataset_path)
data.head()

Ignorando coluna `class`

In [ ]:
X = data.iloc[:, :4]
X.head()

Numero de clusters

In [ ]:
# k = int(input("Nro de clusters (k): "))
k = 3

Silhueta Simplificada

```
Para cada objeto i:
    Encontre o cluster ao qual i pertence
    Calcule a distância média intra-cluster (a_i):
        - Calcule a média das distâncias de i para todos os outros objetos no mesmo cluster
    Calcule a distância média mais próxima inter-cluster (b_i):
        - Para cada cluster diferente do cluster de i:
            - Calcule a média das distâncias de i para todos os objetos no cluster
            - Mantenha o mínimo dessas distâncias
    Calcule o valor da silhueta simplificada para o objeto i:
        - (b_i - a_i) / max(a_i, b_i) se o tamanho do cluster de i for maior que 1, caso contrário, 0
```

In [ ]:
def simplified_silhouette(X, labels):
    n = X.shape[0]
    sil_scores = np.zeros(n)

    for i in range(n):
        # encontra o cluster ao qual i pertence
        own_cluster = labels[i]

        # a_i: distancia media intra-cluster,
        # media das distancias de i para todos os outros objs no mesmo cluster
        a_i = np.mean(np.linalg.norm(X[i] - X[labels == own_cluster], axis=1))

        # b_i: distancia media mais proxima inter-cluster,
        b_i = np.inf
        for label in np.unique(labels):
            if label == own_cluster:
                continue
            # para cada cluster diferente de i
            # 1. calcula a media das distancias de i para todos objetos no cluster
            # 2. calcula o minimo dessas distancias
            b_i = min(b_i, np.mean(np.linalg.norm(X[i] - X[labels == label], axis=1)))

        # calcula o valor da silhueta simplificada para i,
        # se o tamanho do cluster de i for maior que 1, caso contrario 0
        sil_scores[i] = (
            (b_i - a_i) / max(a_i, b_i) if np.sum(labels == own_cluster) > 1 else 0
        )

    return np.mean(sil_scores)

Validacao dos algoritmos

In [ ]:
model = K_Means(k=k)
labels, centroids = model.fit_predict(X.to_numpy())

print("=== K-Means == ")
print("k\t\t", k)
print("dataset\t", dataset_path)
print("Silhouette Score: ", simplified_silhouette(X.to_numpy(), labels))

In [ ]:
ahc = AHC(X, k=k)
ahc.run()
ahc_sil = simplified_silhouette(X.to_numpy(), ahc.labels)

print("=== AHC == ")
print("k\t\t", k)
print("dataset\t", dataset_path)
print("Silhouette Score: ", simplified_silhouette(X.to_numpy(), ahc.labels))